# [Greedy Algorithms, Minimum Spanning Trees, and Dynamic Programming - Week2](https://www.coursera.org/learn/algorithms-greedy/home/welcome)

- outline1
- outline2

## XXI. Kruskal's Minimum Spanning Tree Algorithm

### [Kruskal's MST Algorithm](https://www.coursera.org/learn/algorithms-greedy/lecture/PLdBf/kruskals-mst-algorithm)

#### MST Review

![](https://i.imgur.com/uPpcIex.png)
- 我們有了 Prim's，為何要再學一個 Kruskal's?
    1. 新的 data structure: Union-Find
    1. Kruskal's MST 跟許多 clustering methods 有關聯 (某些 greedy 的 clustering 方法可以視為 Kruskal's MST Algorithm 的變體)
- 這邊一樣假設 $G$ is connected、edge costs 沒有重複。也跟 Prim's 一樣其實不影響正確性，只是方便講解、證明。
- 跟 Prim's MST 一樣，我們在 proof correctness 時需要用到 **Cut Property**，有興趣看 property 證明請見上周 optional video。

#### Example

![](https://i.imgur.com/KlOG4on.png)
- Kruskal's 挑選順序：1 -> 2 -> 3 -> 4(X, becuz of cycle) -> 5 -> 6(X, cycle) -> 7(X, cycle)

#### Kruskal's MST Algorithm

![](https://i.imgur.com/kUQLAxj.png)
- 想法很簡單，把所有 edges sort 過之後，照順序連起來，若會形成 cycle 就跳過
- 注意這邊 rename sorted edges 讓 algorithm 跟 proof 更簡潔

### [Correctness of Kruskal's Algorithm](https://www.coursera.org/learn/algorithms-greedy/lecture/U3ukN/correctness-of-kruskals-algorithm)

#### Correctness of Kruskal (Part I)

![](https://i.imgur.com/DadkPYT.png)
- Part I 我們先只證明 Kruskal 可以 output 出 spanning tree (還不能說 optimal)
- Empty Cut Lemma: 所有的 cut 都有 crossing edges $\iff$ graph is connected (可 review week1)
    - 所以只要證明對於 $T^*$ 來說，所有 cut 都有 crossing edge 即可證明 $T^*$ is connected。
    - 回想 Kruskal，我們把所有 edges 做 sorting 然後一個一個挑，如果會形成 cycle 才不選。因此 cut 一定有一條 crossing edge 會被選到 (by Lonely Cut Corollary，可 review week1)！太神拉
        - 看圖較容易理解

#### Correctness of Kruskal (Part II)

![](https://i.imgur.com/uDg9Dm6.png)
- 要來利用 Cut Property 來證明 minimum 了
    - 假設某個 iteration 已有的 **edge** set 為 $T$，演算法接下來選了 edge $(u,v)$
    - $T\cup\{(u,v)\}$ has no cycle (by algo itself)
    - $\exists$ empty cut $(A,B)$ in $T$ 可以把 $u,v$ 分開，也就是說 $T$ 先前從來沒有在 cut$(A,B)$ 之間連過線
    - 所以 $(u,v)$ 是 algorithm 在 $(A,B)$ 之間連的第一條線，那根據 Kruskal's algo，$(u,v)$ 一定是 cheapest edge between $(A,B)$。
    - 再根據 Cut Property，Q.E.D.
- meta-knowledge: 要分析一個演算法的時候，可以先限縮在某個 iteration 要做某件事的瞬間，較容易分析。
- Q: 利用 Cut Property 有沒有更簡單的證法? 對於每個被選到的 edge $(u,v)$ 我們把那些比 $(u,v)$ 小的 edges 都包起來當 $A$，其餘當 $B$，不就可以利用 Cut Property 嗎?

### [Implementing Kruskal's Algorithm via Union Find I](https://www.coursera.org/learn/algorithms-greedy/lecture/e0TJP/implementing-kruskals-algorithm-via-union-find-i)

#### Kruskal's MST Algorithm

![](https://i.imgur.com/WOgexNa.png)
- sort edges 為何不是 $O(m\log m)$??
    - 因為 $m\le cn^2$ 所以 $O(\log m)$ 可以寫成 $O(\log n)$
    - $m\le cn^2$ 是因為我們假設沒有 parallel edges，就算有，我們 scan 完之後把 cheapest 以外的 parallel edges 都丟掉就好。
- check cycle 用 BFS/DFS 怎麼做@@? 是不是要用 hash table 存 vertices ?
    - 沒這麼複雜啦，我們還有幾個前提可以利用
        1. 我們是要檢查把某個 edge $(u,v)$ 加進去 $T$ 裡面會不會形成 cycle
        2. 我們知道 $T$ 裡面一定沒有 cycle
    - 所以如果加入 $(u,v)$ 就會有 cycle $\iff$ $T$ 裡面有 $u$-$v$ path
        - Q: 這個雙向有證過?
    - 所以**只要去看 $T$ 在 $u$-$v$ 之間有沒有 path 即可**。所以可以用 BFS/DFS 解 $O(n)$。總共 iterate $m$ 次，所以這 loop 總共 $O(mn)$
- 看來 bottleneck 就在 loop 的 $O(mn)$ 了
- 有個更方便的 data structure 可以利用：Union-Find !!! constant time cycle check !!!
    - Q: 嚴格來說應該是 path check 嗎? 只是在這個 case 剛好也可檢查 cycle ?

#### The Union-Find Data Structure

![](https://i.imgur.com/L1mEjKb.png)
- union-find 這類 data structure 存在的理由：能夠區分不同區塊的 object
- 我們對這樣的 data structure 要求不多，只要有兩個 operation 足矣
    - FIND($X$)
    - UNION($C_i,C_j$)
- 為什麼 Kruskal 需要 union operation 呢? 想像現在 $T$ 裡面就是右下那 4 個 connected component，而下一個被挑到的 edge 就是把其中兩個 connected component 連接起來的 edge，這時候兩個 component 就要 union 成一個啦
    - 其實只要 add edge，廣義上來說就是 union 啦
- 需要 find 應該就是為了找 path 吧，如果 $u$ 跟 $v$ 都 find 到同一個 group，那就是有 path！

### [Implementing Kruskal's Algorithm via Union Find II](https://www.coursera.org/learn/algorithms-greedy/lecture/TvDMg/implementing-kruskals-algorithm-via-union-find-ii)

#### Union-Find Basics

![](https://i.imgur.com/6a7OY0K.png)
- Idea #1 + Invariant: 利用 linked structure 把每個 vertices 都指到該 connected component 的 leader vertex 即可 $O(1)$ FIND !
- 所以如果兩個 vertex 找到同一個 leader，那就是同個 connected component，那就是有 path !

#### Maintaining the Invariant

![](https://i.imgur.com/RkH1v2W.png)
- Ans: <font style="opacity:.05">$\theta(n)$</font>
    - 最差狀況就是<font style="opacity:.05">最後一條 edge 要把最後兩個 connected components 連起來的時候。</font>

#### Maintaining the Invariant (con'd)

![](https://i.imgur.com/xdT6t64.png)
- 但照前面說的複雜度，這樣 add 完 edges 就要花費 $O(n^2)$ 的時間。因此我們需要新的 idea 來 implement Union-Find 這樣的 data structure
- Idea #2: merge 的時候只更新較小的 component 的 leader
    - 要做到這點，data structure 要多空間來存每個 component 的 size
- 這時候 update leader 的 worst case 會是如何呢?
    - Ans: 一樣是 $\theta(n)$，只是現在變 $n/2$ 了。看起來還是差不多啊?

#### Updating Leader Pointers
但是！我們換個角度看這件事，let's adopt a vertex-centric view !
- 對某個 vertex 來說，整個 Kruskal + Union-Find(Idea #2) 的過程，它的 leader 會斷斷續續的變動，有時會改，有時保持不變，那麼：
    - ![](https://i.imgur.com/MxPBkF6.png)
    - [ ] $\theta(1)$
    - [ ] $\theta(\log n)$
    - [ ] $\theta(n)$
    - [ ] $\theta(m)$
    - Ans: <font style="opacity:.05">$\theta(\log n)$</font>
        - 因為當 vertex 的 leader 改變了，代表跟它 merge 的 component 比它所在的 component 還要大，所以每次 change leader，大小都會倍增。假設最後 change 了 $k$ 次...
        - ![](https://i.imgur.com/fsMdcf2.png)

#### Running Time of Fast Implementation

![](https://i.imgur.com/8XSvMVW.png)
- 太神拉!!!

### [MSTs: State-of-the-Art and Open Questions [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/Wt9aw/msts-state-of-the-art-and-open-questions-advanced-optional)

## XXII. Clustering 

### [Application to Clustering](https://www.coursera.org/learn/algorithms-greedy/lecture/QWubN/application-to-clustering)

#### Clustering

![](https://i.imgur.com/AiRdbX0.png)
- ML 的 clustering 混進來了 0w0
- (dis)similarity 可以是各式各樣的 metric
    - genome simiarity: 像是之前教到 sequence 的 similarity 可以用 NW-score (用 gap 跟 mismatch 的 penalty 計算出來的 score)
    - Euclidean distance / L1 distance
- 怎麼達成「same cluster = nearby」的目標呢?
    - 做法有很多種！而我們選擇使用 **"optimization-based approach"**，也就是定義一個 objective function，然後想辦法 optimize 它。
        - 我們挑的 function 又稱 k-means objective

#### Max-Spacing k-Clusterings

![](https://i.imgur.com/4RCNCtj.png)
- separated: 任何一對 points 不屬於同個 cluster 就叫 separated
    - 而最該警戒的就是最靠近的 separated points。
- spacing: 就是最靠近的 separated points 的距離。也可以說是 different cluster 之間的最小距離
- problem: 最大化 separated points 的最小距離(或 cluster 之間的(最小)距離)
- 又稱 k-means objective

#### A Greedy Algorithm

![](https://i.imgur.com/i4l8Ah0.png)
- 如果給定一些 clusters 了，那麼要怎麼 greedy 的讓 $\min_{\textrm{separated}p,q}d(p,q)$ 變得更大呢?
- 當然就是每次都把最靠近的 separated points 變成同一個 cluster 啦！
- 有沒有發現這跟 Kruskal's MST algorithm 很像R？只是 clustering 會在找到 $k$ 個 cluster 的時候就先停了。
- 這個 clustering algorithm 有個名稱：single-link clustering。

### [Correctness of Clustering Algorithm](https://www.coursera.org/learn/algorithms-greedy/lecture/7lWTf/correctness-of-clustering-algorithm)

#### Correctness Claim

![](https://i.imgur.com/CWAZQHu.png)

#### Correctness Proof

![](https://i.imgur.com/1xTjHCx.png)

#### Correctness Proof (con'd)

![](https://i.imgur.com/xbPTLrA.png)

## XXIII. Advanced Union Find

### [ Lazy Unions [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/WX3tg/lazy-unions-advanced-optional)

### [Union-by-Rank [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/RF7aH/union-by-rank-advanced-optional)

### [Analysis of Union-by-Rank [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/r2nA3/analysis-of-union-by-rank-advanced-optional)

### [Path Compression [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/Rhh6y/path-compression-advanced-optional)

### [Path Compression: The Hopcroft-Ullman Analysis I [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/KdbbU/path-compression-the-hopcroft-ullman-analysis-i-advanced-optional)

### [Path Compression: The Hopcroft-Ullman Analysis II [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/ttbcu/path-compression-the-hopcroft-ullman-analysis-ii-advanced-optional)

### [The Ackermann Function [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/KZvAe/the-ackermann-function-advanced-optional)

### [Path Compression: Tarjan's Analysis I [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/RqR0G/path-compression-tarjans-analysis-i-advanced-optional)

### [Path Compression: Tarjan's Analysis II [Advanced - Optional]](https://www.coursera.org/learn/algorithms-greedy/lecture/aqa6f/path-compression-tarjans-analysis-ii-advanced-optional)